In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 3 – Grouping, Pivoting, and Combining

## DSC 80, Fall 2023

### Due Date: Monday, October 23rd at 11:59 PM

## Instructions
Welcome to the third lab assignment in DSC 80 this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2023-fa/tree/main).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- **To ensure that all of your work to be submitted is in `lab.py`, we've provided an additional uneditable notebook, called `lab-validation.ipynb`, that contains only the tests and their setup. Make sure you are able to run it top-to-bottom without error before submitting!**
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np

## Part 1: Grouping

Last year, the UK 🇬🇧 announced a new ["High Potential Individual" visa](https://www.lexology.com/library/detail.aspx?g=41fa64ec-9272-468c-bdcb-8002745a754f), which allows graduates of universities ranked in the Top 50 globally to move to the UK without a job lined up. This visa has been a subject of much debate, in part due to how much rankings play a role. (Rest assured, UCSD is on the list!)

In this section, you will analyze a dataset of university rankings, collected from  [here](https://www.kaggle.com/datasets/mylesoneill/world-university-rankings?datasetId=) (though we have pre-processed and modified the original dataset for the purposes of this question). Our version of the dataset is stored in `data/universities_unified.csv`.

Columns:
* `'world_rank'`: world rank of the institution
* `'institution'`: name of the institution
* `'national_rank'`: rank within the nation, formatted as `'country, rank'`
* `'quality_of_education'`: rank by quality of education
* `'alumni_employment'`: rank by alumni employment
* `'quality_of_faculty'`: rank by quality of faculty
* `'publications'`: rank by publications
* `'influence'`: rank by influence
* `'citations'`: rank by number of citations
* `'broad_impact'`: rank by broad impact
* `'patents'`: rank by number of patents
* `'score'`: overall score of the institution, out of 100
* `'control'`: whether the university is public or private
* `'city'`: city in which the institution is located
* `'state'`: state in which the institution is located

### Question 1 – Rankings 1️⃣

There are (still) a few aspects of the dataset we need to clean before it's ready for analysis.

As you work on this question, keep in mind that **you cannot use `for`-loops**. You shouldn't need to – use vectorized `pandas` methods instead.

#### `clean_universities`

Complete the implementation of the function `clean_universities`, which takes in the raw rankings DataFrame and returns a cleaned DataFrame, cleaned according to the following information:

- Some `'institution'` names contain `'\n'` characters (e.g. `'University of California\nSan Diego'`). Replace all instances of `'\n'` with `', '` (a comma and a space) in the `'institution'` column.

- Change the data type of the `'broad_impact'` column to `int`.

* Split `'national_rank'` into two columns, `'nation'` and `'national_rank_cleaned'`, where:
    * `'nation'` is the country (or its dependency) indicated in the first part of `'national_rank'`. 
        * Note that there are **3** countries that appear under different names for different schools. For all 3 of these countries, you should pick **the name that is longer** and use that name for every occurrence of the country. One of the 3 countries is **`'Czech Republic'`**, which also appears as **`'Czechia'`** – since these refer to the same country and `'Czech Republic'` is longer, all instances of either name should be replaced with `'Czech Republic'`. You need to find the other 2 countries on your own. 
        * As is mentioned below, your function will only be tested on the DataFrame in `data/universities_unified.csv`, so you only need to change these 3 country names.
    * `'national_rank_cleaned'` is the integer in the latter part of `'national_rank'`. Make sure that the data type of this column is `int`. 
    * Don't include the original `'national_rank'` column in the output DataFrame.
* Create a Boolean column `'is_r1_public'`. This column should contain `True` if a university is public and classified as R1 and `False` otherwise. Treat `np.NaN`s as False. **Note that in the raw DataFrame, a university is classified as R1 if and only if it has non-null values in all of the following columns: `'control'`, `'city'`, and `'state'`.**
    - Read [this page](https://en.wikipedia.org/wiki/List_of_research_universities_in_the_United_States) to learn more about what it means for a university to be classified as R1.
    
**The only dataset your function will be tested on is `data/universities_unified.csv`; you don't need to worry about other hidden test sets.** In addition, please return a *copy* of the original DataFrame; don't modify the original.

<br>

Now, we can do some basic exploration.

#### `university_info`

Complete the implementation of the function `university_info`, which takes in the **cleaned** DataFrame outputted by `clean_universities` and returns the following values in a list:
* Among `'state(s)'` with three or more `'institution(s)'` in the dataset, the `'state'` whose universities have the lowest mean `'score'`.
* The proportion of the `'institution(s)'` in the top 100 for which the `'quality of faculty'` ranking is also in the top 100.
* The number of `'state(s)'` where at least 50% of the `'institution(s)'` are private (i.e. have an `'is_r1_public'` of `False`).
* The lowest ranking `'institution'`, according to `'world_rank'`, that is ranked #1 in its nation (i.e. that has a `'national_rank_cleaned'` of 1).

You can assume there are no ties.

In [176]:
def clean_universities(df):
    
    res = df.copy()
    #replace /n in institution
    res['institution'] = res['institution'].str.replace('\n', ', ')
    
    #Change the data type of the 'broad_impact' column to int
    res['broad_impact']= res['broad_impact'].astype(int)
    
    #Split 'national_rank' into two columns!!!!!!
    res[['nation','national_rank_cleaned']] = res.national_rank.str.split(", ",expand=True) 
    res = res.drop(columns=['national_rank'])
    res['nation'] = res['nation'].replace({
        'Czechia': 'Czech Republic',
        'UK': 'United Kingdom',
        'USA': 'United States'
    })
    res['national_rank_cleaned'] = res['national_rank_cleaned'].astype(int)
    
    
    res['is_r1_public'] = ((res['control'].notna()) & (res['city'].notna()) & (res['state'].notna()) & (res['control'] == 'Public'))
    res['is_r1_public'].fillna(False, inplace=True)  
    
    return res

In [654]:
cleaned

,world_rank,institution,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,control,city,state,nation,national_rank_cleaned,is_r1_public
0,1,Harvard University,1,1,1,1,1,1,1,3,100.00,Private (non-profit),Cambridge,MA,United States,1,False
1,2,Stanford University,9,2,4,5,3,3,4,10,98.66,Private (non-profit),Stanford,CA,United States,2,False
2,3,Massachusetts Institute of Technology,3,11,2,15,2,2,2,1,97.54,Private (non-profit),Cambridge,MA,United States,3,False
3,4,University of Cambridge,2,10,5,11,6,12,13,48,96.81,NaN,NaN,NaN,United Kingdom,1,False
4,5,University of Oxford,7,13,10,7,12,7,9,15,96.46,NaN,NaN,NaN,United Kingdom,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,University of the Algarve,367,567,218,926,845,812,969,816,44.03,NaN,NaN,NaN,Portugal,7,False
996,997,Alexandria University,236,566,218,997,908,645,981,871,44.03,NaN,NaN,NaN,Egypt,4,False
997,998,Federal University of Ceará,367,549,218,830,823,812,975,824,44.03,NaN,NaN,NaN,Brazil,18,False
998,999,University of A Coruña,367,567,218,886,974,812,975,651,44.02,NaN,NaN,NaN,Spain,40,False


In [585]:
def university_info(cleaned):
    lowest_mean_state = (cleaned.groupby('state')
            .filter(lambda df: df['institution'].count() >= 3)
            .groupby('state')['score']
            .mean().idxmin())
    
    proportions_within_100 = (cleaned[(cleaned['world_rank'] <= 100) 
                                      & (cleaned['quality_of_faculty'] <=100)].shape[0] /100)
    
    percentage = cleaned.groupby('state')['is_r1_public'].mean()
    num_state_50 = int(percentage[percentage <= 0.5].count())
    
    national_1 = cleaned[cleaned['national_rank_cleaned'] == 1].set_index('institution')
    lowest_world_rank = national_1['world_rank'].sort_values(ascending=False).idxmax()
    
    return [lowest_mean_state, proportions_within_100, num_state_50, lowest_world_rank]

In [586]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') /'universities_unified.csv'
df = pd.read_csv(fp)
cleaned = clean_universities(df)
info = university_info(cleaned)

In [587]:
info

['AL', 0.71, 13, 'University of Bucharest']

In [588]:
cleaned

,world_rank,institution,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,control,city,state,nation,national_rank_cleaned,is_r1_public
0,1,Harvard University,1,1,1,1,1,1,1,3,100.00,Private (non-profit),Cambridge,MA,United States,1,False
1,2,Stanford University,9,2,4,5,3,3,4,10,98.66,Private (non-profit),Stanford,CA,United States,2,False
2,3,Massachusetts Institute of Technology,3,11,2,15,2,2,2,1,97.54,Private (non-profit),Cambridge,MA,United States,3,False
3,4,University of Cambridge,2,10,5,11,6,12,13,48,96.81,NaN,NaN,NaN,United Kingdom,1,False
4,5,University of Oxford,7,13,10,7,12,7,9,15,96.46,NaN,NaN,NaN,United Kingdom,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,University of the Algarve,367,567,218,926,845,812,969,816,44.03,NaN,NaN,NaN,Portugal,7,False
996,997,Alexandria University,236,566,218,997,908,645,981,871,44.03,NaN,NaN,NaN,Egypt,4,False
997,998,Federal University of Ceará,367,549,218,830,823,812,975,824,44.03,NaN,NaN,NaN,Brazil,18,False
998,999,University of A Coruña,367,567,218,886,974,812,975,651,44.02,NaN,NaN,NaN,Spain,40,False


In [219]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – High Standards ™️ 

At various points in this question, you'll need to compute the standard deviation. As we saw in [Lecture 3](https://dsc80.com/resources/lectures/lec03/lec03.html#Aside:-std), `numpy` and `pandas` use different formulas by default to calculate the standard deviation; throughout, make sure you use the `ddof=0` argument when calculating standard deviations. In general, if you're working with the entire population, you should use `ddof = 0`, and if you're working with a sample, you should use `ddof = 1`.


#### `std_scores_by_nation` 

Complete the implementation of the function `std_scores_by_nation`, which takes in a **cleaned** DataFrame, like the one returned by `clean_universities`, and outputs a DataFrame: 
- with the same rows as the input, 
- with three columns: `'institution'`, `'nation'`, and `'score'` (in that order),
- where the `'score'` column is **standardized** by `'nation'` - that is, the `'score'`s for each country are converted to standard units, using the mean and standard deviation of the `'score'`s for that country. If a `'score'` is `np.NaN`, leave it as `np.NaN`.

<br>

#### `su_and_spread`

Lastly, complete the implementation of the function `su_and_spread`, which returns the answers to the following two questions, as a list.

****Part 1****

Let's compare rankings between two countries – the US 🇺🇸 and Canada 🇨🇦. There are in total $n$ universities in the US and $m$ universities in Canada. Suppose $x_1, x_2, ..., x_n$ are the `'world_rank'`s for US universities in **increasing order**, meaning that $x_1$ is the `'world_rank'` of the "best" US university. Similarly, $y_1, y_2, ..., y_m$ are the `'world_rank'`s for Canadian universities, also in increasing order. 

Suppose we take the aforementioned `'world_rank'`s and sort them together in **increasing order**, e.g. $x_1, x_2, y_1, x_3, ...$. **We define $R$ to be the average of the positions of the $x$ values.**

For example, if there are 3 US universities (so $n=3$) and 2 Canadian universities ($m=2$), and
  
$$x_1 = 1, x_2 = 3, x_3 = 10, \:\:\:\: y_1 = 5, y_2 = 15$$

When we sort the rankings in increasing order, we'd get 1, 3, 5, 10, 15, which correspond to the values $x_1, x_2, y_1, x_3, y_2$. The $x$ values are at positions 1, 2, and 4. Then, $R = \frac{1 + 2 + 4}{3} = \frac{7}{3}$. (Note that this is **not** the average of 1, 3, and 10).


**Question:** If we believe that US universities in general rank higher than Canadian universities, should $R$ be
1. larger than $\frac{m + n}{2}$?
2. smaller than $\frac{m + n}{2}$?
3. equal to $\frac{m + n}{2}$?


Store your answer – either 1, 2, or 3 – in the first element of `su_and_spread`'s output list. Note that this is a classical example of a non-parametric hypothesis test called a rank test.

<br>

****Part 2****

Which `'nation'` has the largest variation in `'score'`s before standardization? Store your answer in the second element of `su_and_spread`'s output list.

In [323]:
def std_scores_by_nation(cleaned):
    stdized = cleaned.groupby('nation')['score'].transform(lambda x: (x - x.mean()) / x.std(ddof=0))
    out = cleaned[['institution', 'nation']].assign(score=stdized)
    return out

In [324]:
def su_and_spread():
    return [2, 'United States']

In [325]:
aaa=cleaned[(cleaned['nation'] == 'United States') | (cleaned['nation'] == 'Canada')]
aaa.reset_index(drop=True, inplace=True)
aaa.sort_values(by='world_rank',ascending=True)
us_schools = aaa.loc[aaa['nation'] == 'United States']
positions = sum(us_schools.index+1) /aaa.shape[0]
positions

112.3587786259542

In [326]:
aaa.shape[0]/2

131.0

In [327]:
nation_std_dev = cleaned.groupby('nation')['score'].std(ddof=0)
nation_with_largest_variation = nation_std_dev.idxmax()
nation_with_largest_variation

'United States'

In [328]:
# do not edit this cell -- it is needed for the tests
fp = Path('data') / 'universities_unified.csv'
universities = pd.read_csv(fp)
cleaned = clean_universities(universities)
universities_out = std_scores_by_nation(cleaned)
su_and_spread_out = su_and_spread()

In [589]:
universities_out

,institution,nation,score
0,Harvard University,United States,4.637401
1,Stanford University,United States,4.512844
2,Massachusetts Institute of Technology,United States,4.408737
3,University of Cambridge,United Kingdom,5.250795
4,University of Oxford,United Kingdom,5.213140
...,...,...,...
995,University of the Algarve,Portugal,-1.144479
996,Alexandria University,Egypt,-0.721110
997,Federal University of Ceará,Brazil,-0.495751
998,University of A Coruña,Spain,-0.775241


In [329]:
grader.check("q2")

q2 results: All test cases passed!

## Part 2: Combining Data

### Question 3 – Making Connections 🤝

A group of students decided to send out a survey to their connections on LinkedIn. Each student asks 1000 of their connections for their first and last name, the company they currently work at, their job title, their email, and the university they attended.

**Your job is to combine all the data contained in the files `survey*.csv` (stored within the `data/responses` folder) into a single DataFrame. The number of files and the number of rows in each file may vary, so don't hardcode your answers!** To do so, implement the following two functions.

#### `read_linkedin_survey`

Complete the implementation of the function `read_linkedin_survey`, which takes in a string describing the path to a folder containing `survey*.csv` files and outputs a DataFrame with six columns titled `'first name'`, `'last name'`, `'current company'`, `'job title'`, `'email'`, and `'university'` (in that order) containing the survey information for all files combined. Make sure to reset the index of the combined DataFrame before returning it so that the index is unique. 

***Hints***:

- Take a look at a few of the files in the `responses` folder. You may have to do some data cleaning to combine the DataFrames!

- You can list the files in a directory using `.iterdir()`.

***Note***: Remember to use `Path()` to build file paths.

<br>

#### `com_stats`

Complete the implementation of the function `com_stats`, which takes in a DataFrame returned by `read_linkedin_survey` and returns a list containing, in the following order: 
- The proportion of people who went to a university with the string `'Ohio'` in its name that have the string `'Nurse'` somewhere in their job title.
- The number of job titles that **end** with the string `'Engineer'`, not the number of people.
- The job title that has the longest name (there are no ties).
- The number of people who have the word `'manager'` in their job title, uppercase or lowercase (`'Manager'`, `'manager'`, and `'mANAgeR'` should all count).

In [339]:
def read_linkedin_survey(dirname):
    
    files = [os.path.join(dirname, file) for file in os.listdir(dirname)]
    df_list = [pd.read_csv(file) for file in files]

    for df in df_list:
        df.columns = df.columns.str.lower().str.replace('_', ' ')

    df = pd.concat(df_list, sort=True)
    df = df[['first name', 'last name', 'current company', 'job title', 'email', 'university']]
    df = df.reset_index(drop=True)
    
    return df

In [347]:
df=read_linkedin_survey(dirname)

In [425]:
def com_stats(df):
    ohios= df[df['university'].str.contains('Ohio', case=False, na = False)]
    ohio_nurse_prop = (ohios.loc[(ohios['job title'].str.contains('Nurse', case=False, na = False))].shape[0] / ohios.shape[0])
    
    endwith_engineer = df[df['job title'].str.endswith('Engineer', na=False)]['job title'].nunique()
    
    longest_title = df['job title'][df['job title'].str.len().idxmax()]
    
    manager_count = df.loc[df['job title'].str.contains('manager', case=False, na=False)].shape[0]
    
    return [ohio_nurse_prop, endwith_engineer, longest_title, manager_count]

In [590]:
q3_out

,first name,last name,current company,job title,email,university
0,Consuelo,Pippin,Mayert and Sons,Dental Hygienist,cpippin0@wordpress.org,Military University Shoumen
1,Amata,Gotts,Padberg and Sons,Civil Engineer,agotts1@posterous.com,Abo Akademi University
2,Glori,Warmisham,Bruen-Rosenbaum,Senior Developer,gwarmisham2@techcrunch.com,St. Vincent College
3,Byron,Lytle,Upton Inc,Developer IV,blytle3@businessweek.com,"University of the West Indies, Mona"
4,Carolee,Swadlin,Swift-Lemke,Librarian,cswadlin4@drupal.org,Novosibirsk State University
...,...,...,...,...,...,...
4995,Rebeka,Chedzoy,Zieme Inc,Software Consultant,rchedzoyrn@opera.com,Walden University
4996,Troy,Akehurst,"Bednar, Hettinger and Mills",Analyst Programmer,takehurstro@zdnet.com,Sardar Patel University
4997,Elihu,Collecott,Lesch-Hermann,GIS Technical Architect,NaN,St. Petersburg State University of Civil Aviation
4998,Loree,Landes,Koch-Stracke,Analyst Programmer,llandesrq@netlog.com,University of Maryland at College Park


In [427]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'responses'
q3_out = read_linkedin_survey(dirname)
stats_out = com_stats(q3_out)
stats_out

[0.23076923076923078, 13, 'Business Systems Development Analyst', 369]

In [428]:
grader.check("q3")

q3 results: All test cases passed!

### Question 4 – Survey Says... 👨‍👩‍👧‍👦

Professor Karthikeya often sends out extra credit surveys asking students for their favorite animals, movies, and other favorite things. These surveys are stored in the `data/extra-credit-surveys` folder. Each file in that folder corresponds to a different survey question (except for `favorite1.csv`, which contains students' names and IDs).

Here's how extra credit works:
- Each student who has completed at least 50% of the survey questions receives 5 points of extra credit.
- If there is at least one survey question that at least 80% of the class answered (e.g. favorite animal), **everyone** in the class receives 1 point of extra credit. This overall class extra credit only applies twice, so if for example 95% of students answer the favorite color survey question and 91% answer the favorite animal survey question, and and 97% answer the favorite movie question, the entire class still receives 2 extra point as a class, not 3.
- Note that this means that the most extra credit any student can earn is 7 points.

#### `read_student_surveys`

Complete the implementation of the function `read_student_surveys` which takes in a string describing the path to a folder containing `favorite*.csv` files and outputs a DataFrame containing all of the survey data combined, indexed by student ID (a value 1-1000).

***Note***: Remember to use `Path()` to build file paths.


<br>

#### `check_credit`

Complete the implementation of the function `check_credit` which takes in a DataFrame returned by `read_student_surveys` and outputs a DataFrame indexed by student ID (a value 1-1000) with two columns:
- `'name'`, containing the name of each student, and
- `'ec'`, containing the number of extra credit points each student earned.

In [592]:
def read_student_surveys(dirname):
    
    files = [os.path.join(dirname, file) for file in os.listdir(dirname)]
    file_list = [pd.read_csv(f, index_col='id') for f in files]
    combined = pd.concat(file_list, axis=1, sort=True)

    return combined

In [593]:
read_student_surveys(dirname)

,movie,genre,name,animal,plant,color
id,,,,,,
1,NaN,(no genres listed),Myrtia,Long-crested hawk eagle,NaN,Red
2,NaN,Documentary,Nathanil,Euro wallaby,NaN,Khaki
3,"Glass-blower's Children, The (Glasblåsarns barn)",NaN,Joni,Brown brocket,NaN,Red
4,NaN,(no genres listed),Prentice,"Peccary, white-lipped",NaN,Yellow
5,NaN,NaN,Claudette,"Capuchin, brown",NaN,Fuscia
...,...,...,...,...,...,...
996,Kung Phooey!,Horror|Mystery|Sci-Fi,Addie,"Eland, common",NaN,Purple
997,Angel Heart,NaN,Valaria,Agouti,NaN,Blue
998,NaN,NaN,Gunilla,"Shelduck, european",NaN,NaN


In [709]:
def check_credit(df):
    ec = ((df.drop('name', axis=1).notnull().mean(axis=1) >= 0.5)
          .replace({False: 0, True: 5}))

    ec += min(((df.drop('name', axis=1).notnull().mean() >= 0.8).any().astype(int)),2)
        
    out = pd.concat([df[['name']], ec], axis=1, sort=True)
    
    out.columns = ['name', 'ec']
    
    return out

In [713]:
check_credit_out


,name,0
id,,
1,Myrtia,6
2,Nathanil,6
3,Joni,6
4,Prentice,6
5,Claudette,1
...,...,...
996,Addie,6
997,Valaria,6
998,Gunilla,1


In [711]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data')/'extra-credit-surveys'
q4_out = read_student_surveys(dirname)
check_credit_out = check_credit(q4_out)

In [712]:
grader.check("q4")

q4 results:
    q4 - 1 result:
        Test case passed!

    q4 - 2 result:
        Test case passed!

    q4 - 3 result:
        Test case passed!

    q4 - 4 result:
        Test case passed!

    q4 - 5 result:
        Trying:
            check_credit_out['ec'].max() == 6
        Expecting:
            True
        **********************************************************************
        Line 1, in q4 4
        Failed example:
            check_credit_out['ec'].max() == 6
        Exception raised:
            Traceback (most recent call last):
              File "/Users/mingjunsun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3621, in get_loc
                return self._engine.get_loc(casted_key)
              File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
              File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
              File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
              File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
            KeyError: 'ec'

            The above exception was the direct cause of the following exception:

            Traceback (most recent call last):
              File "/Users/mingjunsun/opt/anaconda3/lib/python3.9/doctest.py", line 1334, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q4 4[0]>", line 1, in <module>
                check_credit_out['ec'].max() == 6
              File "/Users/mingjunsun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py", line 3505, in __getitem__
                indexer = self.columns.get_loc(key)
              File "/Users/mingjunsun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3623, in get_loc
                raise KeyError(key) from err
            KeyError: 'ec'

### Question 5 – Paw Patrol 🐾

You are analyzing data from a veterinarian clinic. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, available procedures, and procedure history. The column names are self-explanatory. These DataFrames are provided to you:
-  `owners` stores the customer information, where every `'OwnerID'` is unique (verify this yourself).
-  `pets` stores the pet information. Each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records. Most procedures were given to a pet in `pets`.

Complete the implementation of the following three functions, which each ask you to answer a specific question.

#### `most_popular_procedure`

What is the most popular `'ProcedureType'` amongst all pets in the `pets` DataFrame? Complete the implementation of the function `most_popular_procedure`, which takes in two DataFrames, `pets` and `procedure_history`, and returns the name of the most popular `'ProcedureType'` as a string.

Note that some pets are registered but haven't had any procedures performed. Also, some pets that have had procedures done are not registered in `pets`.

<br>

#### `pet_name_by_owner`

What is the name of each customer's pet(s)? Complete the implementation of the function `pet_name_by_owner`, which takes in two DataFrames, `owners` and `pets`, and returns a Series whose index contains owner first names, and whose values are pet names as **strings**. If an owner has multiple pets, the value corresponding to that owner should instead be a **list of pet names as strings**.

Note that owner first names are not necessarily unique, and so the Series you return will not necessarily have a unique index.

<br>

#### `total_cost_per_city`

Note that the `owners` DataFrame has a `'City'` column, describing the city in which each pet owner and their pets live. How much did each city spend in total on procedures? Complete the implementation of the function `total_cost_per_city`, which takes in four DataFrames, `owners`, `pets`, `procedure_history`, and `procedure_detail`, and returns a Series indexed by `'City'` that describes the total amount that each city has spent on pets' procedures.

***Hint:*** At some point, you may have to merge on multiple columns.

In [641]:
def most_popular_procedure(pets, procedure_history):
    merged = pets.merge(right=procedure_history, on='PetID')
    return merged['ProcedureType'].value_counts().idxmax()

In [696]:
def pet_name_by_owner(owners, pets):
    pets = pets.rename(columns = {'Name': 'Petname'})
    merged_df = owners.merge(pets,how='left', on='OwnerID')
    res = (merged_df.groupby(['Name', 'Surname'])['Petname']
           .agg(lambda x: list(x) if len(x) > 1 else x.iloc[0])
          )
    
    return res

In [702]:
out_02.loc['John']

Name
John      Kashi
John    Biscuit
John      Rumba
Name: Petname, dtype: object

In [646]:
def total_cost_per_city(owners, pets, procedure_history, procedure_detail):
    pets = pets.rename(columns = {'Name': 'Petname'})
    pet_owners=owners.merge(right=pets,how='left', on='OwnerID')
    pet_owners_history=pet_owners.merge(right=procedure_history,how='left', on='PetID')
    pet_owners_history_detail=pet_owners_history.merge(right=procedure_detail, how='left', on=['ProcedureType', 'ProcedureSubCode'])
    return pet_owners_history_detail.groupby('City')['Price'].sum()

In [657]:
# do not edit this cell -- it is needed for the tests
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

out_01 = most_popular_procedure(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [661]:
out_03

City
Ann Arbor               450.0
Bloomfield Township       0.0
Bridgman                  0.0
Bruce Crossing            0.0
Caro                      0.0
Center Line              10.0
Clam River                0.0
Commerce                 10.0
Davison                   0.0
Detroit                 305.0
Dutton                    0.0
East Lansing             40.0
Farmington                0.0
Farmington Hills         10.0
Flint                    15.0
Fremont                   0.0
Fruitport                 0.0
Grand Ledge               0.0
Grand Rapids           1240.0
Highland Park             0.0
Hudsonville               0.0
Kalamazoo                10.0
Lansing                  30.0
Lincoln Park              0.0
Livonia                  10.0
Marquette                50.0
Michigan Center          10.0
Middleville               0.0
Oshtemo                   0.0
Plymouth                 10.0
Pontiac                  30.0
Rochester Hills           0.0
Roseville                10.0
Sagin

In [648]:
grader.check("q5")

q5 results: All test cases passed!

## Part 3: Pivot Tables

### Question 6 – Summarizing Sales 💰

Recall from [Lecture 5](https://dsc80.com/resources/lectures/lec05/lec05.html), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few simple problems using the `pivot_table` method. 

**We have provided outlines for the DataFrames you need to create, but yours may have a different number of rows and columns and different values.**

**`for loops` are not allowed in this question.**

#### `average_seller`

Complete the implementation of the function `average_seller`, which takes in the `sales` DataFrame and returns a DataFrame that contains the average sales for each seller, indexed by `'Name'` and containing the column `'Average Sales'`. There should not be any `NaN`s.

***Note:*** You may be able to implement `average_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Average Sales</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Complete the implementation of the function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each product, indexed by `'Name'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Product</th>
      <th>boat</th>
      <th>book</th>
      <th>hotel</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Complete the implementation of the function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `total_by_month`

Complete the implementation of the function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>


***Note:*** [Here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.

<span style="color:red"> ***Important:***</span> You can't use loop by any means in this problem!

In [618]:
sales

,Name,Product,Date,Total,Month
0,Smith,book,01.01.2012,200,January
1,Jones,pen,02.20.2013,300,February
2,Trump,hotel,03.03.2015,100,March
3,Smith,book,05.10.2013,2000,May
4,Jones,book,02.20.2013,30,February
5,Trump,boat,03.30.2017,700,March
6,Smith,ruler,07.05.2014,2100,July
7,Jones,ruler,02.25.2015,350,February
8,Trump,book,03.03.2015,1000,March
9,Smith,pen,01.01.2012,2500,January


In [619]:
def average_seller(sales):
    sales = sales.copy().fillna(0)
    series=sales.groupby('Name')['Total'].mean().astype(int)
    res=pd.DataFrame({'Average Sales':series})
    return res

In [620]:
def product_name(sales):
    pivot=sales.pivot_table(
    index='Name',
    columns='Product',
    values='Total', 
    aggfunc='count')
    return pivot

In [621]:
def count_product(sales):
    return sales.pivot_table(index=['Product', 'Name'], 
                  columns='Date', 
                  values='Total', 
                  aggfunc='sum',
                 fill_value=0)

In [615]:
def total_by_month(sales):
    dates = sales["Date"]
    dates = pd.to_datetime(dates, format='%m.%d.%Y')
    months = dates.dt.month_name()
    sales['Month'] = months
    return sales.pivot_table(index=['Name', 'Product'], 
                  columns='Month', 
                  values='Total', 
                  aggfunc='sum',
                 fill_value=0)

In [623]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'sales.csv'
sales = pd.read_csv(fp)
q6_average_seller_out = average_seller(sales)
q6_product_name_out = product_name(sales)
q6_product_count_out = count_product(sales)
q6_total_by_month_out = total_by_month(sales)
q6_average_seller_out

,Average Sales
Name,
Jones,920
Smith,1700
Trump,487


In [617]:
q6_product_name_out

Product,boat,book,hotel,pen,ruler
Name,,,,,
Jones,NaN,1.0,NaN,1.0,2.0
Smith,NaN,2.0,NaN,1.0,1.0
Trump,1.0,1.0,1.0,1.0,NaN


In [612]:
q6_product_count_out

Date           01.01.2012  02.20.2013  02.25.2015  03.03.2015  03.30.2017  \
Product Name                                                                
boat    Trump           0           0           0           0         700   
book    Jones           0          30           0           0           0   
        Smith         200           0           0           0           0   
        Trump           0           0           0        1000           0   
hotel   Trump           0           0           0         100           0   
pen     Jones           0         300           0           0           0   
        Smith        2500           0           0           0           0   
        Trump           0           0           0         150           0   
ruler   Jones           0        3000         350           0           0   
        Smith           0           0           0           0           0   

Date           05.10.2013  07.05.2014  
Product Name                           
boat    Trump           0           0  
book    Jones           0           0  
        Smith        2000           0  
        Trump           0           0  
hotel   Trump           0           0  
pen     Jones           0           0  
        Smith           0           0  
        Trump           0           0  
ruler   Jones           0           0  
        Smith           0        2100

In [606]:
q6_total_by_month_out

Month          February  January  July  March   May
Name  Product                                      
Jones book           30        0     0      0     0
      pen           300        0     0      0     0
      ruler        3350        0     0      0     0
Smith book            0      200     0      0  2000
      pen             0     2500     0      0     0
      ruler           0        0  2100      0     0
Trump boat            0        0     0    700     0
      book            0        0     0   1000     0
      hotel           0        0     0    100     0
      pen             0        0     0    150     0

In [557]:
grader.check("q6")

q6 results: All test cases passed!

## Congratulations! You're done with Lab 3! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To verify that all of your work is indeed in `lab.py`, and that you didn't accidentally implement a function in this notebook and not in `lab.py`, we've included another notebook in the lab folder, called `lab-validation.ipynb`. `lab-validation.ipynb` is a version of this notebook with only the `grader.check` cells and the code needed to set up the tests. 

### **Go to `lab-validation.ipynb`, and go to Kernel > Restart & Run All.** This will check if all `grader.check` test cases pass using just the code in `lab.py`.

Once you're able to pass all test cases in `lab-validation.ipynb`, including the call to `grader.check_all()` at the very bottom, then you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [627]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!

q6 results: All test cases passed!